## **Setup**

In [ ]:
from google.colab import drive
mount_path = '/content/gdrive/'
drive.mount(mount_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

#hide
!pip install utils
from utils import *

     |████████████████████████████████| 727kB 6.2MB/s 
     |████████████████████████████████| 1.0MB 20.1MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 358kB 33.5MB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
     |████████████████████████████████| 40kB 5.6MB/s 
     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 2.6MB 31.8MB/s 


In [ ]:
from fastai.text.all import *

In [ ]:
import os
import shutil
import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory

from sklearn.metrics import f1_score, accuracy_score

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

In [ ]:
# Set and test path to competition data files
competition_path = 'My Drive/AI For Good - AI Blitz 3/FNEWS/Data/'
model_path='My Drive/AI For Good - AI Blitz 3/FNEWS/Models/'
filename = 'train.zip'
assert(os.path.exists(f'{mount_path}/{competition_path}/{filename}'))
print('Drive mounted correctly and data accessible')

Drive mounted correctly and data accessible


# Unzip and combine dataframes
Combine the train and validation sets from the competition organizers, we're going to use our own!


In [ ]:
home_dir='/content/All_Data/'

# All_Data is the home directory for the images, from which training and validation splits will be taken
if not os.path.exists(home_dir):
  os.mkdir(home_dir)

  for filename in ['train','val','test']:
    with ZipFile(f'{mount_path}/{competition_path}/{filename}.zip', 'r') as zip_ref:
      zip_ref.extractall(home_dir)    

In [ ]:
testdf=pd.read_csv(f'{home_dir}test.csv')
print(testdf.shape)

(115999, 1)


In [ ]:
results_path='My Drive/AI For Good - AI Blitz 3/FNEWS/Results/'

In [ ]:
model_names=['200904_1415_AWD_LSTM_cv0','200904_1521_AWD_LSTM_cv1']

for model_name in model_names:
  learn=load_learner(f'{mount_path}{model_path}{model_name}.pkl')

  tstdls=learn.dls.test_dl(testdf)

  preds=learn.get_preds(dl=tstdls)[0].numpy()

  np.save(f'{mount_path}{results_path}{model_name}.npy',preds)

In [ ]:
res1=np.load(f'{mount_path}{results_path}200904_1415_AWD_LSTM_cv0.npy')
res2=np.load(f'{mount_path}{results_path}200904_1415_AWD_LSTM_cv0.npy')

In [ ]:
combres=(res1+res2)/2

In [ ]:
combres=pd.Series(np.argmax(combres,axis=1)).map({0:'fake',1:'real'})

In [ ]:
np.mean(combres=='fake')

0.2902438814127708

In [ ]:
datestr=datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%y%m%d_%H%M")
combres.to_csv(datestr+'_submission.csv',header=['label'],index=False)